In [1]:
from requests import get
import numpy as np
import pandas as pd
import json
from pprint import pprint
%load_ext autotime

In [101]:
api_key = "1dbab546719d8085308a5bfde18381d7"

time: 1.23 ms


In [40]:
all_tasks = get("https://www.openml.org/api/v1/json/task/list?api_key=" + api_key).json()
supervised_classification_task_ids = []
for task in all_tasks['tasks']['task']:
    if task['task_type'] == 'Supervised Classification':
        supervised_classification_task_ids.append(task['task_id'])

In [141]:
num_tasks = len(all_task_ids)
counter = 0
supervised_classification_flow_ids = []
for task_id in supervised_classification_task_ids:
    counter += 1
    if counter % 25 == 0:
        print("{}% complete".format(round(counter/num_tasks,3)*100))
    try:
        runs = get("https://www.openml.org/api/v1/json/run/list/task/" + str(task_id) + "/limit/10000?api_key=" + api_key)
        runs = runs.json()
    except:
        runs = json.loads(runs.text.replace('\n', ''))
    if 'error' not in runs:
        for run in runs['runs']['run']:
            supervised_classification_flow_ids.append(run['flow_id'])
        supervised_classification_flow_ids = list(set(supervised_classification_flow_ids))

0.9208103130755065% complete
1.841620626151013% complete
2.7624309392265194% complete
3.683241252302026% complete
4.6040515653775325% complete
5.524861878453039% complete
6.445672191528545% complete
7.366482504604052% complete
8.287292817679557% complete
9.208103130755065% complete
10.128913443830571% complete
11.049723756906078% complete
11.970534069981584% complete
12.89134438305709% complete
13.812154696132598% complete
14.732965009208105% complete
15.65377532228361% complete
16.574585635359114% complete
17.495395948434624% complete
18.41620626151013% complete
19.337016574585636% complete
20.257826887661142% complete
21.178637200736645% complete
22.099447513812155% complete
23.02025782688766% complete
23.941068139963168% complete
24.861878453038674% complete
25.78268876611418% complete
26.703499079189687% complete
27.624309392265197% complete
28.5451197053407% complete
29.46593001841621% complete
30.386740331491712% complete
31.30755064456722% complete
32.22836095764273% complete
33

In [188]:
num_flows = len(supervised_classification_flow_ids[1:])
counter = 0
supervised_classification_flow_descriptions = []
for flow in supervised_classification_flow_ids[1:]:
    counter += 1
    if counter % 25 == 0:
        print("{}% complete".format(round(counter/num_flows,3)*100))
    flow_description = get("https://www.openml.org/api/v1/json/flow/" + str(flow) + "?api_key=" + api_key).json()
    if 'error' not in flow_description:
        supervised_classification_flow_descriptions.append(flow_description)

0.8641548565502939% complete
1.7283097131005878% complete
2.5924645696508817% complete
3.4566194262011756% complete
4.320774282751469% complete
5.184929139301763% complete
6.049083995852056% complete
6.913238852402351% complete
7.777393708952644% complete
8.641548565502937% complete
9.505703422053232% complete
10.369858278603527% complete
11.23401313515382% complete
12.098167991704113% complete
12.962322848254408% complete
13.826477704804702% complete
14.690632561354994% complete
15.554787417905288% complete
16.418942274455585% complete
17.283097131005874% complete
18.14725198755617% complete
19.011406844106464% complete
19.875561700656757% complete
20.739716557207053% complete
21.603871413757343% complete
22.46802627030764% complete
23.332181126857936% complete
24.196335983408225% complete
25.060490839958522% complete
25.924645696508815% complete
26.788800553059104% complete
27.652955409609405% complete
28.517110266159694% complete
29.381265122709987% complete
30.245419979260284% comp

In [45]:
sklearn_flows = []
for flow in supervised_classification_flow_descriptions:
    if flow['flow']['name'][0:7] == 'sklearn':
        sklearn_flows.append(flow['flow']['id'])

time: 6.08 ms


In [72]:
num_flows = len(sklearn_flows)
counter = 0
sklearn_flow_descriptions = {}
sklearn_run_descriptions = {}
for flow in sklearn_flows:
    counter += 1
    if counter % 5 == 0:
        print("{}% complete".format(round(counter/num_flows,3)*100))
    run_descriptions = get("https://www.openml.org/api/v1/json/run/list/flow/" + flow + "?api_key=" + api_key).json()
    flow_description = get("https://www.openml.org/api/v1/json/flow/" + flow + "?api_key=" + api_key).json()
    if ('error' not in run_descriptions) and ('error' not in flow_description):
        sklearn_run_descriptions[flow] = run_descriptions
        sklearn_flow_descriptions[flow] = flow_description

1.858736059479554% complete
3.717472118959108% complete
5.5762081784386615% complete
7.434944237918216% complete
9.293680297397769% complete
11.152416356877323% complete
13.011152416356877% complete
14.869888475836431% complete
16.728624535315987% complete
18.587360594795538% complete
20.44609665427509% complete
22.304832713754646% complete
24.1635687732342% complete
26.022304832713754% complete
27.881040892193308% complete
29.739776951672862% complete
31.59851301115242% complete
33.457249070631974% complete
35.31598513011153% complete
37.174721189591075% complete
39.03345724907063% complete
40.89219330855018% complete
42.75092936802974% complete
44.60966542750929% complete
46.468401486988846% complete
48.3271375464684% complete
50.18587360594795% complete
52.04460966542751% complete
53.90334572490706% complete
55.762081784386616% complete
57.62081784386617% complete
59.479553903345725% complete
61.33828996282528% complete
63.19702602230484% complete
65.05576208178438% complete
66.9144

In [ ]:
with open('sklearn_flow_descriptions.json', 'w') as outfile:
    json.dump(sklearn_flow_descriptions, outfile)
with open('sklearn_run_descriptions.json', 'w') as outfile:
    json.dump(sklearn_run_descriptions, outfile)

In [78]:
sklearn_task_ids = []
sklearn_setup_ids = []
for flow,runs in sklearn_run_descriptions.items():
    for run in runs['runs']['run']:
        sklearn_task_ids.append(run['task_id'])
        sklearn_setup_ids.append(run['setup_id'])
sklearn_task_ids = list(set(sklearn_task_ids))
sklearn_setup_ids = list(set(sklearn_setup_ids))

time: 58.8 ms


In [84]:
num_tasks = len(sklearn_task_ids)
counter = 0
sklearn_task_descriptions = {}
for task_id in sklearn_task_ids:
    counter += 1
    if counter % 10 == 0:
        print("{}% complete".format(round(counter/num_tasks,3)*100))
    task_description = get("https://www.openml.org/api/v1/json/task/" + str(task_id) + "?api_key=" + api_key).json()
    sklearn_task_descriptions[task_id] = task_description

1.4204545454545454% complete
2.840909090909091% complete
4.261363636363636% complete
5.681818181818182% complete
7.1022727272727275% complete
8.522727272727272% complete
9.943181818181818% complete
11.363636363636363% complete
12.784090909090908% complete
14.204545454545455% complete
15.625% complete
17.045454545454543% complete
18.46590909090909% complete
19.886363636363637% complete
21.306818181818183% complete
22.727272727272727% complete
24.147727272727273% complete
25.568181818181817% complete
26.988636363636363% complete
28.40909090909091% complete
29.829545454545453% complete
31.25% complete
32.67045454545455% complete
34.090909090909086% complete
35.51136363636363% complete
36.93181818181818% complete
38.35227272727273% complete
39.77272727272727% complete
41.19318181818182% complete
42.61363636363637% complete
44.034090909090914% complete
45.45454545454545% complete
46.875% complete
48.29545454545455% complete
49.715909090909086% complete
51.13636363636363% complete
52.5568181

In [ ]:
with open('sklearn_task_descriptions.json', 'w') as outfile:
    json.dump(sklearn_task_descriptions, outfile)

In [154]:
num_setups = len(sklearn_setup_ids)
counter = 0
sklearn_setup_descriptions = {}
for setup_id in sklearn_setup_ids:
    counter += 1
    if counter % 100 == 0:
        print("{}% complete".format(round(counter/num_setups,3)*100))
    setup_description = get("https://www.openml.org/api/v1/json/setup/" + str(setup_id) + "?api_key=" + api_key).json()
    sklearn_setup_descriptions[setup_id] = setup_description

0.8% complete
1.5% complete
2.3% complete
3.1% complete
3.8% complete
4.6% complete
5.4% complete
6.1% complete
6.9% complete
7.7% complete
8.4% complete
9.2% complete
10.0% complete
10.7% complete
11.5% complete
12.3% complete
13.0% complete
13.8% complete
14.6% complete
15.4% complete
16.1% complete
16.900000000000002% complete
17.7% complete
18.4% complete
19.2% complete
20.0% complete
20.7% complete
21.5% complete
22.3% complete
23.0% complete
23.799999999999997% complete
24.6% complete
25.3% complete
26.1% complete
26.900000000000002% complete
27.6% complete
28.4% complete
29.2% complete
29.9% complete
30.7% complete
31.5% complete
32.2% complete
33.0% complete
33.800000000000004% complete
34.5% complete
35.3% complete
36.1% complete
36.8% complete
37.6% complete
38.4% complete
39.1% complete
39.900000000000006% complete
40.699999999999996% complete
41.5% complete
42.199999999999996% complete
43.0% complete
43.8% complete
44.5% complete
45.300000000000004% complete
46.1% complete


In [155]:
with open('sklearn_setup_descriptions.json', 'w') as outfile:
    json.dump(sklearn_setup_descriptions, outfile)

time: 14.5 s


In [143]:
sklearn_data_set_ids = []
for task_id,task_description in sklearn_task_descriptions.items():
    data_set_id = task_description['task']['input'][0]['data_set']['data_set_id']
    sklearn_data_set_ids.append(data_set_id)

time: 4.6 ms


In [146]:
num_data_sets = len(sklearn_data_set_ids)
counter = 0
sklearn_data_set_descriptions = {}
for data_set_id in sklearn_data_set_ids:
    counter += 1
    if counter % 10 == 0:
        print("{}% complete".format(round(counter/num_data_sets,3)*100))
    data_set_description = get("https://www.openml.org/api/v1/json/data/" + str(data_set_id) + "?api_key=" + api_key).json()
    sklearn_data_set_descriptions[data_set_id] = data_set_description

1.4000000000000001% complete
2.8000000000000003% complete
4.3% complete
5.7% complete
7.1% complete
8.5% complete
9.9% complete
11.4% complete
12.8% complete
14.2% complete
15.6% complete
17.0% complete
18.5% complete
19.900000000000002% complete
21.3% complete
22.7% complete
24.099999999999998% complete
25.6% complete
27.0% complete
28.4% complete
29.799999999999997% complete
31.2% complete
32.7% complete
34.1% complete
35.5% complete
36.9% complete
38.4% complete
39.800000000000004% complete
41.199999999999996% complete
42.6% complete
44.0% complete
45.5% complete
46.9% complete
48.3% complete
49.7% complete
51.1% complete
52.6% complete
54.0% complete
55.400000000000006% complete
56.8% complete
58.199999999999996% complete
59.699999999999996% complete
61.1% complete
62.5% complete
63.9% complete
65.3% complete
66.8% complete
68.2% complete
69.6% complete
71.0% complete
72.39999999999999% complete
73.9% complete
75.3% complete
76.7% complete
78.10000000000001% complete
79.5% complete

In [147]:
with open('sklearn_data_set_descriptions.json', 'w') as outfile:
    json.dump(sklearn_data_set_descriptions, outfile)

time: 79.1 ms


In [5]:
with open('flow_descriptions.json') as data_file:
    flow_descriptions = json.load(data_file)

time: 821 ms
